# Twitch data analysis

Dataset nodi (streamer):
- numero persone totali raggiunte
- tempo streammato
- viewer medi
- aggiornamento community (persone totali/tempo)/viewer medi
- peak viewers
- categorie streammate
- link twitter
- dati twitter

Dataset relazioni/ matrice streamer:
- percentuale
- numero assoluto

#### Setup parameters

In [1]:
freq = 2               #Sample frequency (1=5min, 2=10min, 3=15min ecc.)
min_stream_time = 3    #Minimum live time parameter
min_watch_time = 3     #Minimum threshold of time as a spectator (min_watch_time <= min_stream_time)
min_num_spect_mean = 3 #Minimum number of spectator

print(f'Minimum live time threshold:\t\t\t {5 * freq * min_stream_time} min')
print(f'Minimum threshold of time as a spectator:\t {5 * freq * min_watch_time} min')

Minimum live time threshold:			 30 min
Minimum threshold of time as a spectator:	 30 min


#### Import libraries

In [2]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
from pandas import json_normalize
from collections import Counter
from collections import defaultdict

## Nodes dataset

### Import data

Complete list of files

In [3]:
completelist = [os.path.join(path, name) for path, subdirs, files in os.walk("./files_stream/") for name in files]
print("Number of files:", len(completelist))
print(*completelist[:5], sep = "\n")

Number of files: 274
./files_stream/2022-05-06_00-04.json
./files_stream/2022-05-06_00-09.json
./files_stream/2022-05-06_00-14.json
./files_stream/2022-05-06_00-19.json
./files_stream/2022-05-06_00-24.json


Sampling files based on frequency

In [4]:
flist = completelist[::freq]
print("Number of files:", len(flist))
print(*flist[:5], sep = "\n")

Number of files: 137
./files_stream/2022-05-06_00-04.json
./files_stream/2022-05-06_00-14.json
./files_stream/2022-05-06_00-24.json
./files_stream/2022-05-06_00-34.json
./files_stream/2022-05-06_00-44.json


Import selected files

In [5]:
data_dict = {}

for file in flist:
    start = './files_stream/'
    end = '.json'
    time = file[len(start): - len(end)]
    time_obj = datetime.strptime(time, '%Y-%m-%d_%H-%M') #extract timestamp
    
    with open(file, 'r') as f:
        data_dict[time_obj] = json.load(f)

print("Number of keys (time intervals):", len(data_dict.keys()))

Number of keys (time intervals): 137


Example `'pasquale890': {'spect': ['anytow'], 'game_name': 'FIFA 22', 'viewer_count': 0}`


In [6]:
bot_list = pd.read_csv('bot_list.csv')
bot_list = bot_list['Twitch Username'].tolist()
bot_list[:10]

['turbopascai',
 'tteyyd',
 'twitchdetails',
 '0liviajadeee',
 'judgejudysiayer',
 'thelurkertv',
 'nothingbutgay',
 'streamers_on_discord',
 '0_applebadapple_0',
 'agressive_sock']

### Remove bot

Difference between viewer and spect for a time instant, can be negative and positive:

- bot excluded from viewer count
- viewer not connected in chat
- data updated at different instants

In [7]:
spect_with_bot = {}

for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect_count'] = len(data_dict[time_obj][streamer]['spect'])
    data_dict[time_obj][streamer]['diff_count'] = data_dict[time_obj][streamer]['viewer_count'] - data_dict[time_obj][streamer]['spect_count']
    spect_with_bot[streamer] = len(data_dict[time_obj][streamer]['spect'])
    '''
    print(f"{streamer}\n\
    \tViewer Count: {data_dict[time_obj][streamer]['viewer_count']}\
    \tSpect Count: {spect_with_bot[streamer]}\
    \tDifference: {data_dict[time_obj][streamer]['diff_count']}\
    ")
    '''

In [8]:
def remove_bot(spect_with_bot, bot_set):
    return list(set(spect_with_bot) - bot_set)

In [9]:
bot_set = set(bot_list)
for time_obj in data_dict.keys():
    for streamer in data_dict[time_obj].keys():
        data_dict[time_obj][streamer]['spect'] = remove_bot(data_dict[time_obj][streamer]['spect'], bot_set)

Check again the difference between viewer and spect

In [10]:
for streamer in data_dict[time_obj].keys():
    data_dict[time_obj][streamer]['spect_count'] = len(data_dict[time_obj][streamer]['spect'])
    data_dict[time_obj][streamer]['diff_count'] = data_dict[time_obj][streamer]['viewer_count'] - data_dict[time_obj][streamer]['spect_count']
    
    print(f"{streamer}\n\
    \tViewer Count: {data_dict[time_obj][streamer]['viewer_count']}\
    \tSpect Count: {len(data_dict[time_obj][streamer]['spect'])}\
    \tDifference: {data_dict[time_obj][streamer]['diff_count']}\
    \tBot Number: {spect_with_bot[streamer] - len(data_dict[time_obj][streamer]['spect'])}\
    ")

Tumblurr
    	Viewer Count: 15777    	Spect Count: 11886    	Difference: 3891    	Bot Number: 12    
DarioMocciaTwitch
    	Viewer Count: 9906    	Spect Count: 7950    	Difference: 1956    	Bot Number: 14    
ZanoXVII
    	Viewer Count: 9462    	Spect Count: 6393    	Difference: 3069    	Bot Number: 16    
ilGabbrone
    	Viewer Count: 6248    	Spect Count: 4334    	Difference: 1914    	Bot Number: 15    
Homyatol
    	Viewer Count: 4102    	Spect Count: 3309    	Difference: 793    	Bot Number: 15    
Moonryde
    	Viewer Count: 3826    	Spect Count: 2757    	Difference: 1069    	Bot Number: 14    
TheRealMarzaa
    	Viewer Count: 3332    	Spect Count: 2493    	Difference: 839    	Bot Number: 15    
ocwsport
    	Viewer Count: 3098    	Spect Count: 2253    	Difference: 845    	Bot Number: 12    
double_face_twich
    	Viewer Count: 2087    	Spect Count: 238    	Difference: 1849    	Bot Number: 14    
ilpengwin
    	Viewer Count: 2061    	Spect Count: 1106    	Difference: 955    	Bot Nu

    	Viewer Count: 4    	Spect Count: 1    	Difference: 3    	Bot Number: 6    
tauro17_
    	Viewer Count: 3    	Spect Count: 2    	Difference: 1    	Bot Number: 1    
9rdose
    	Viewer Count: 3    	Spect Count: 2    	Difference: 1    	Bot Number: 3    
ilcalabres1
    	Viewer Count: 3    	Spect Count: 2    	Difference: 1    	Bot Number: 7    
selly_brixia
    	Viewer Count: 3    	Spect Count: 2    	Difference: 1    	Bot Number: 3    
VecchiNabbi
    	Viewer Count: 3    	Spect Count: 0    	Difference: 3    	Bot Number: 8    
Barbabietoli
    	Viewer Count: 3    	Spect Count: 1    	Difference: 2    	Bot Number: 5    
Aurol_Sol
    	Viewer Count: 3    	Spect Count: 1    	Difference: 2    	Bot Number: 10    
therealksmoke
    	Viewer Count: 3    	Spect Count: 2    	Difference: 1    	Bot Number: 4    
brojllo
    	Viewer Count: 3    	Spect Count: 1    	Difference: 2    	Bot Number: 8    
Masa_shoku
    	Viewer Count: 3    	Spect Count: 0    	Difference: 3    	Bot Number: 3    
Jigen997
 

gabbo_yt666
    	Viewer Count: 1    	Spect Count: 1    	Difference: 0    	Bot Number: 5    
Avior89
    	Viewer Count: 1    	Spect Count: 0    	Difference: 1    	Bot Number: 5    
amurgo21
    	Viewer Count: 1    	Spect Count: 0    	Difference: 1    	Bot Number: 1    
salvinosalvo
    	Viewer Count: 1    	Spect Count: 0    	Difference: 1    	Bot Number: 1    
Dark1000097
    	Viewer Count: 1    	Spect Count: 0    	Difference: 1    	Bot Number: 4    
melovik1988
    	Viewer Count: 1    	Spect Count: 0    	Difference: 1    	Bot Number: 6    
Kira_1992
    	Viewer Count: 1    	Spect Count: 1    	Difference: 0    	Bot Number: 5    
fsc_senna
    	Viewer Count: 1    	Spect Count: 1    	Difference: 0    	Bot Number: 5    
g1nomotorin0
    	Viewer Count: 1    	Spect Count: 0    	Difference: 1    	Bot Number: 5    
ITALUKE
    	Viewer Count: 1    	Spect Count: 0    	Difference: 1    	Bot Number: 8    
pierluigi73
    	Viewer Count: 1    	Spect Count: 0    	Difference: 1    	Bot Number: 1    
D

### Obtain streamer list and prop

In [11]:
streamer_repeated = []
game_name_repeated = []
viewer_count_repeated = []
spect_count_repeated = []
spect_repeated = []

for istante, value in data_dict.items():
    streamer_repeated.extend(list(value.keys()))
    for streamer, val in value.items():
        game_name_repeated.append(val['game_name'])
        viewer_count_repeated.append(val['viewer_count'])
        spect_count_repeated.append(len(val['spect']))
        spect_repeated.append(val['spect'])

len(streamer_repeated)

193495

In [12]:
streamer_repeated = pd.DataFrame({'streamer': streamer_repeated,
                                  'game_name': game_name_repeated,
                                  'viewer_count': viewer_count_repeated,
                                  'spect_count': spect_count_repeated,
                                  'spect': spect_repeated
                                 })
streamer_repeated

streamer                     game_name  viewer_count  \
0                GrenBaud                 Just Chatting         26220   
1           TheRealMarzaa                         Slots         19473   
2              ilGabbrone                Virtual Casino          6673   
3       DarioMocciaTwitch                         Spore          5549   
4                ocwsport                        Sports          5359   
...                   ...                           ...           ...   
193490           bossgv93  Call Of Duty: Modern Warfare             0   
193491        pasquale890                       FIFA 22             0   
193492        giannekkttv        Call of Duty: Vanguard             0   
193493          djalexr93  Call Of Duty: Modern Warfare             0   
193494      CrazyyyKill3r            Grand Theft Auto V             0   

        spect_count                                              spect  
0             19878  [pietro6666666, marcolino4567, zrunaa, teo_mar...  
1             14563  [cris1234567890124, m4tt1a_fn, sbucione27, col...  
2              4656  [st3f4n010, tentayson, tommasook_, slymanfredo...  
3              4480  [manumrch, abster_t, neisha_yeager, rzambio, w...  
4              3767  [bellapaco, lollinhoc, matty__22__, kebho97, j...  
...             ...                                                ...  
193490            0                                                 []  
193491            0                                                 []  
193492            0                                                 []  
193493            0                                                 []  
193494            1                                         [antovm__]  

[193495 rows x 5 columns]

In [13]:
streamer_grouped = streamer_repeated.groupby("streamer").agg({'viewer_count': ['mean', 'max', 'count'],
                                                              'spect_count': 'mean',
                                                              'game_name': (lambda x: list(set(x))),
                                                              'spect': 'sum'
                                                             })

In [14]:
streamer_grouped.columns = ["_".join(pair) for pair in streamer_grouped.columns]
streamer_grouped['spect_sum'] = streamer_grouped['spect_sum'].apply(set)
streamer_grouped['spect_count_tot'] = streamer_grouped['spect_sum'].apply(len)

streamer_grouped.sort_values('spect_count_tot', ascending = False, inplace = True)
streamer_grouped

viewer_count_mean  viewer_count_max  viewer_count_count  \
streamer                                                                  
ZanoXVII             10777.040000             13747                  25   
TheRealMarzaa        12135.633333             22115                  30   
Tumblurr             13510.857143             16339                  14   
ilGabbrone            4591.880952              7856                  42   
Xiuder_               3137.527778              6383                  72   
...                           ...               ...                 ...   
faze_lemonss             1.333333                 3                  18   
favigarcia17             1.000000                 1                   1   
favelles                 1.000000                 1                  15   
faustopaolillo           1.000000                 1                  11   
zzz_shadow               1.000000                 1                   7   

                spect_count_mean  \
streamer                           
ZanoXVII             8184.560000   
TheRealMarzaa        9313.566667   
Tumblurr            11122.928571   
ilGabbrone           3238.357143   
Xiuder_              2146.291667   
...                          ...   
faze_lemonss            0.000000   
favigarcia17            0.000000   
favelles                0.000000   
faustopaolillo          0.000000   
zzz_shadow              0.000000   

                                               game_name_<lambda>  \
streamer                                                            
ZanoXVII                         [FIFA 22, Sports, Just Chatting]   
TheRealMarzaa                   [Minecraft, Slots, Just Chatting]   
Tumblurr        [Just Chatting, Fishing Sim World: Bass Pro Sh...   
ilGabbrone                        [Virtual Casino, Just Chatting]   
Xiuder_                                                [Fortnite]   
...                                                           ...   
faze_lemonss                                           [Fortnite]   
favigarcia17                              [Call of Duty: Warzone]   
favelles                                  [Call of Duty: Warzone]   
faustopaolillo                     [Call Of Duty: Modern Warfare]   
zzz_shadow                                [Call of Duty: Warzone]   

                                                        spect_sum  \
streamer                                                            
ZanoXVII        {az0990ale, gianvix_99, chappyfn1, mec_ranger,...   
TheRealMarzaa   {cris1234567890124, zanno96, gianvix_99, ilmit...   
Tumblurr        {cris1234567890124, ilmiticocomodino, gianvix_...   
ilGabbrone      {michele2k1, dea_tv, luciusconoscoi, marcolino...   
Xiuder_         {ctvilingdarc, mec_ranger, mrpopogaina, lupo_b...   
...                                                           ...   
faze_lemonss                                                   {}   
favigarcia17                                                   {}   
favelles                                                       {}   
faustopaolillo                                                 {}   
zzz_shadow                                                     {}   

                spect_count_tot  
streamer                         
ZanoXVII                  49984  
TheRealMarzaa             46263  
Tumblurr                  31578  
ilGabbrone                28935  
Xiuder_                   24364  
...                         ...  
faze_lemonss                  0  
favigarcia17                  0  
favelles                      0  
faustopaolillo                0  
zzz_shadow                    0  

[13691 rows x 7 columns]

In [15]:
streamer_grouped_cut = streamer_grouped.drop(streamer_grouped[(streamer_grouped.viewer_count_count < min_stream_time)
                                                              | (streamer_grouped.spect_count_tot < min_num_spect_mean)
                                                              | (streamer_grouped.viewer_count_mean < min_num_spect_mean)
                                                             ].index)
streamer_grouped_cut#.sort_values('spect_count_tot', ascending = False)

viewer_count_mean  viewer_count_max  viewer_count_count  \
streamer                                                                  
ZanoXVII             10777.040000             13747                  25   
TheRealMarzaa        12135.633333             22115                  30   
Tumblurr             13510.857143             16339                  14   
ilGabbrone            4591.880952              7856                  42   
Xiuder_               3137.527778              6383                  72   
...                           ...               ...                 ...   
LorenzoFDM               3.000000                 6                   9   
Sic_7358                 3.500000                 5                   4   
TheFiveFellas            4.842105                 7                  19   
NRAnonimoPlays           4.181818                 5                  11   
JesterDas                4.727273                 6                  11   

                spect_count_mean  \
streamer                           
ZanoXVII             8184.560000   
TheRealMarzaa        9313.566667   
Tumblurr            11122.928571   
ilGabbrone           3238.357143   
Xiuder_              2146.291667   
...                          ...   
LorenzoFDM              0.888889   
Sic_7358                2.750000   
TheFiveFellas           0.894737   
NRAnonimoPlays          2.181818   
JesterDas               1.909091   

                                               game_name_<lambda>  \
streamer                                                            
ZanoXVII                         [FIFA 22, Sports, Just Chatting]   
TheRealMarzaa                   [Minecraft, Slots, Just Chatting]   
Tumblurr        [Just Chatting, Fishing Sim World: Bass Pro Sh...   
ilGabbrone                        [Virtual Casino, Just Chatting]   
Xiuder_                                                [Fortnite]   
...                                                           ...   
LorenzoFDM                         [Call Of Duty: Modern Warfare]   
Sic_7358                                  [Call of Duty: Warzone]   
TheFiveFellas                                    [SMITE, FIFA 22]   
NRAnonimoPlays                                         [Warframe]   
JesterDas                                     [League of Legends]   

                                                        spect_sum  \
streamer                                                            
ZanoXVII        {az0990ale, gianvix_99, chappyfn1, mec_ranger,...   
TheRealMarzaa   {cris1234567890124, zanno96, gianvix_99, ilmit...   
Tumblurr        {cris1234567890124, ilmiticocomodino, gianvix_...   
ilGabbrone      {michele2k1, dea_tv, luciusconoscoi, marcolino...   
Xiuder_         {ctvilingdarc, mec_ranger, mrpopogaina, lupo_b...   
...                                                           ...   
LorenzoFDM         {ohmkiller_23, toninouspiderman, mastraccio92}   
Sic_7358                            {alexwlf46, luppio87, hjess_}   
TheFiveFellas                     {huntik11, killeenpax, emdotem}   
NRAnonimoPlays                 {fhplayer45, yollo987, nr_caustic}   
JesterDas                {greensandal, soulrichard01, francix_05}   

                spect_count_tot  
streamer                         
ZanoXVII                  49984  
TheRealMarzaa             46263  
Tumblurr                  31578  
ilGabbrone                28935  
Xiuder_                   24364  
...                         ...  
LorenzoFDM                    3  
Sic_7358                      3  
TheFiveFellas                 3  
NRAnonimoPlays                3  
JesterDas                     3  

[3976 rows x 7 columns]

## Relations dataset

In [16]:
spect_dict = streamer_grouped_cut[['spect_sum']].to_dict()['spect_sum']
len(spect_dict.keys())

3976

In [17]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger('Progress')

viewerOverlapDict = {}
completedStreamers = set() #Save which streamers have been processed to avoid repeating
count = 1

for key in spect_dict:
    tempList = {}
    totalLength = len(spect_dict.keys())
    logger.info(str(count) + "/" + str(totalLength)) #Print progress so I can keep track

    for comparisonKey in spect_dict: #Loop through every key again for each key in the dictionary
        if(comparisonKey != key and comparisonKey not in completedStreamers): #If its not a self comparison and the comparison hasn't already been completed
            overlapSize = len(spect_dict[key] & spect_dict[comparisonKey]) #Find the overlap size of the two streamers using set intersection
            if len(spect_dict[key]) < len(spect_dict[comparisonKey]):
                tempList[comparisonKey] = round(overlapSize/len(spect_dict[key])*100, 2) #If the size is over 300 add {comparisonStreamer: overlap} to the dictionary
            else:
                tempList[comparisonKey] = round(overlapSize/len(spect_dict[comparisonKey])*100, 2)
    viewerOverlapDict[key] = tempList #Add this comparison dictionary to the larger dictionary for that streamer
    #completedStreamers.add(key) #Add the streamer to completed as no comparisons using this streamer need to be done anymore
    count+=1

INFO:Progress:1/3976
INFO:Progress:2/3976
INFO:Progress:3/3976
INFO:Progress:4/3976
INFO:Progress:5/3976
INFO:Progress:6/3976
INFO:Progress:7/3976
INFO:Progress:8/3976
INFO:Progress:9/3976
INFO:Progress:10/3976
INFO:Progress:11/3976
INFO:Progress:12/3976
INFO:Progress:13/3976
INFO:Progress:14/3976
INFO:Progress:15/3976
INFO:Progress:16/3976
INFO:Progress:17/3976
INFO:Progress:18/3976
INFO:Progress:19/3976
INFO:Progress:20/3976
INFO:Progress:21/3976
INFO:Progress:22/3976
INFO:Progress:23/3976
INFO:Progress:24/3976
INFO:Progress:25/3976
INFO:Progress:26/3976
INFO:Progress:27/3976
INFO:Progress:28/3976
INFO:Progress:29/3976
INFO:Progress:30/3976
INFO:Progress:31/3976
INFO:Progress:32/3976
INFO:Progress:33/3976
INFO:Progress:34/3976
INFO:Progress:35/3976
INFO:Progress:36/3976
INFO:Progress:37/3976
INFO:Progress:38/3976
INFO:Progress:39/3976
INFO:Progress:40/3976
INFO:Progress:41/3976
INFO:Progress:42/3976
INFO:Progress:43/3976
INFO:Progress:44/3976
INFO:Progress:45/3976
INFO:Progress:46/39

INFO:Progress:362/3976
INFO:Progress:363/3976
INFO:Progress:364/3976
INFO:Progress:365/3976
INFO:Progress:366/3976
INFO:Progress:367/3976
INFO:Progress:368/3976
INFO:Progress:369/3976
INFO:Progress:370/3976
INFO:Progress:371/3976
INFO:Progress:372/3976
INFO:Progress:373/3976
INFO:Progress:374/3976
INFO:Progress:375/3976
INFO:Progress:376/3976
INFO:Progress:377/3976
INFO:Progress:378/3976
INFO:Progress:379/3976
INFO:Progress:380/3976
INFO:Progress:381/3976
INFO:Progress:382/3976
INFO:Progress:383/3976
INFO:Progress:384/3976
INFO:Progress:385/3976
INFO:Progress:386/3976
INFO:Progress:387/3976
INFO:Progress:388/3976
INFO:Progress:389/3976
INFO:Progress:390/3976
INFO:Progress:391/3976
INFO:Progress:392/3976
INFO:Progress:393/3976
INFO:Progress:394/3976
INFO:Progress:395/3976
INFO:Progress:396/3976
INFO:Progress:397/3976
INFO:Progress:398/3976
INFO:Progress:399/3976
INFO:Progress:400/3976
INFO:Progress:401/3976
INFO:Progress:402/3976
INFO:Progress:403/3976
INFO:Progress:404/3976
INFO:Progre

INFO:Progress:719/3976
INFO:Progress:720/3976
INFO:Progress:721/3976
INFO:Progress:722/3976
INFO:Progress:723/3976
INFO:Progress:724/3976
INFO:Progress:725/3976
INFO:Progress:726/3976
INFO:Progress:727/3976
INFO:Progress:728/3976
INFO:Progress:729/3976
INFO:Progress:730/3976
INFO:Progress:731/3976
INFO:Progress:732/3976
INFO:Progress:733/3976
INFO:Progress:734/3976
INFO:Progress:735/3976
INFO:Progress:736/3976
INFO:Progress:737/3976
INFO:Progress:738/3976
INFO:Progress:739/3976
INFO:Progress:740/3976
INFO:Progress:741/3976
INFO:Progress:742/3976
INFO:Progress:743/3976
INFO:Progress:744/3976
INFO:Progress:745/3976
INFO:Progress:746/3976
INFO:Progress:747/3976
INFO:Progress:748/3976
INFO:Progress:749/3976
INFO:Progress:750/3976
INFO:Progress:751/3976
INFO:Progress:752/3976
INFO:Progress:753/3976
INFO:Progress:754/3976
INFO:Progress:755/3976
INFO:Progress:756/3976
INFO:Progress:757/3976
INFO:Progress:758/3976
INFO:Progress:759/3976
INFO:Progress:760/3976
INFO:Progress:761/3976
INFO:Progre

INFO:Progress:1073/3976
INFO:Progress:1074/3976
INFO:Progress:1075/3976
INFO:Progress:1076/3976
INFO:Progress:1077/3976
INFO:Progress:1078/3976
INFO:Progress:1079/3976
INFO:Progress:1080/3976
INFO:Progress:1081/3976
INFO:Progress:1082/3976
INFO:Progress:1083/3976
INFO:Progress:1084/3976
INFO:Progress:1085/3976
INFO:Progress:1086/3976
INFO:Progress:1087/3976
INFO:Progress:1088/3976
INFO:Progress:1089/3976
INFO:Progress:1090/3976
INFO:Progress:1091/3976
INFO:Progress:1092/3976
INFO:Progress:1093/3976
INFO:Progress:1094/3976
INFO:Progress:1095/3976
INFO:Progress:1096/3976
INFO:Progress:1097/3976
INFO:Progress:1098/3976
INFO:Progress:1099/3976
INFO:Progress:1100/3976
INFO:Progress:1101/3976
INFO:Progress:1102/3976
INFO:Progress:1103/3976
INFO:Progress:1104/3976
INFO:Progress:1105/3976
INFO:Progress:1106/3976
INFO:Progress:1107/3976
INFO:Progress:1108/3976
INFO:Progress:1109/3976
INFO:Progress:1110/3976
INFO:Progress:1111/3976
INFO:Progress:1112/3976
INFO:Progress:1113/3976
INFO:Progress:11

INFO:Progress:1415/3976
INFO:Progress:1416/3976
INFO:Progress:1417/3976
INFO:Progress:1418/3976
INFO:Progress:1419/3976
INFO:Progress:1420/3976
INFO:Progress:1421/3976
INFO:Progress:1422/3976
INFO:Progress:1423/3976
INFO:Progress:1424/3976
INFO:Progress:1425/3976
INFO:Progress:1426/3976
INFO:Progress:1427/3976
INFO:Progress:1428/3976
INFO:Progress:1429/3976
INFO:Progress:1430/3976
INFO:Progress:1431/3976
INFO:Progress:1432/3976
INFO:Progress:1433/3976
INFO:Progress:1434/3976
INFO:Progress:1435/3976
INFO:Progress:1436/3976
INFO:Progress:1437/3976
INFO:Progress:1438/3976
INFO:Progress:1439/3976
INFO:Progress:1440/3976
INFO:Progress:1441/3976
INFO:Progress:1442/3976
INFO:Progress:1443/3976
INFO:Progress:1444/3976
INFO:Progress:1445/3976
INFO:Progress:1446/3976
INFO:Progress:1447/3976
INFO:Progress:1448/3976
INFO:Progress:1449/3976
INFO:Progress:1450/3976
INFO:Progress:1451/3976
INFO:Progress:1452/3976
INFO:Progress:1453/3976
INFO:Progress:1454/3976
INFO:Progress:1455/3976
INFO:Progress:14

...

INFO:Progress:2441/3976
INFO:Progress:2442/3976
INFO:Progress:2443/3976
INFO:Progress:2444/3976
INFO:Progress:2445/3976
INFO:Progress:2446/3976
INFO:Progress:2447/3976
INFO:Progress:2448/3976
INFO:Progress:2449/3976
INFO:Progress:2450/3976
INFO:Progress:2451/3976
INFO:Progress:2452/3976
INFO:Progress:2453/3976
INFO:Progress:2454/3976
INFO:Progress:2455/3976
INFO:Progress:2456/3976
INFO:Progress:2457/3976
INFO:Progress:2458/3976
INFO:Progress:2459/3976
INFO:Progress:2460/3976
INFO:Progress:2461/3976
INFO:Progress:2462/3976
INFO:Progress:2463/3976
INFO:Progress:2464/3976
INFO:Progress:2465/3976
INFO:Progress:2466/3976
INFO:Progress:2467/3976
INFO:Progress:2468/3976
INFO:Progress:2469/3976
INFO:Progress:2470/3976
INFO:Progress:2471/3976
INFO:Progress:2472/3976
INFO:Progress:2473/3976
INFO:Progress:2474/3976
INFO:Progress:2475/3976
INFO:Progress:2476/3976
INFO:Progress:2477/3976
INFO:Progress:2478/3976
INFO:Progress:2479/3976
INFO:Progress:2480/3976
INFO:Progress:2481/3976
INFO:Progress:24

INFO:Progress:2783/3976
INFO:Progress:2784/3976
INFO:Progress:2785/3976
INFO:Progress:2786/3976
INFO:Progress:2787/3976
INFO:Progress:2788/3976
INFO:Progress:2789/3976
INFO:Progress:2790/3976
INFO:Progress:2791/3976
INFO:Progress:2792/3976
INFO:Progress:2793/3976
INFO:Progress:2794/3976
INFO:Progress:2795/3976
INFO:Progress:2796/3976
INFO:Progress:2797/3976
INFO:Progress:2798/3976
INFO:Progress:2799/3976
INFO:Progress:2800/3976
INFO:Progress:2801/3976
INFO:Progress:2802/3976
INFO:Progress:2803/3976
INFO:Progress:2804/3976
INFO:Progress:2805/3976
INFO:Progress:2806/3976
INFO:Progress:2807/3976
INFO:Progress:2808/3976
INFO:Progress:2809/3976
INFO:Progress:2810/3976
INFO:Progress:2811/3976
INFO:Progress:2812/3976
INFO:Progress:2813/3976
INFO:Progress:2814/3976
INFO:Progress:2815/3976
INFO:Progress:2816/3976
INFO:Progress:2817/3976
INFO:Progress:2818/3976
INFO:Progress:2819/3976
INFO:Progress:2820/3976
INFO:Progress:2821/3976
INFO:Progress:2822/3976
INFO:Progress:2823/3976
INFO:Progress:28

INFO:Progress:3125/3976
INFO:Progress:3126/3976
INFO:Progress:3127/3976
INFO:Progress:3128/3976
INFO:Progress:3129/3976
INFO:Progress:3130/3976
INFO:Progress:3131/3976
INFO:Progress:3132/3976
INFO:Progress:3133/3976
INFO:Progress:3134/3976
INFO:Progress:3135/3976
INFO:Progress:3136/3976
INFO:Progress:3137/3976
INFO:Progress:3138/3976
INFO:Progress:3139/3976
INFO:Progress:3140/3976
INFO:Progress:3141/3976
INFO:Progress:3142/3976
INFO:Progress:3143/3976
INFO:Progress:3144/3976
INFO:Progress:3145/3976
INFO:Progress:3146/3976
INFO:Progress:3147/3976
INFO:Progress:3148/3976
INFO:Progress:3149/3976
INFO:Progress:3150/3976
INFO:Progress:3151/3976
INFO:Progress:3152/3976
INFO:Progress:3153/3976
INFO:Progress:3154/3976
INFO:Progress:3155/3976
INFO:Progress:3156/3976
INFO:Progress:3157/3976
INFO:Progress:3158/3976
INFO:Progress:3159/3976
INFO:Progress:3160/3976
INFO:Progress:3161/3976
INFO:Progress:3162/3976
INFO:Progress:3163/3976
INFO:Progress:3164/3976
INFO:Progress:3165/3976
INFO:Progress:31

INFO:Progress:3467/3976
INFO:Progress:3468/3976
INFO:Progress:3469/3976
INFO:Progress:3470/3976
INFO:Progress:3471/3976
INFO:Progress:3472/3976
INFO:Progress:3473/3976
INFO:Progress:3474/3976
INFO:Progress:3475/3976
INFO:Progress:3476/3976
INFO:Progress:3477/3976
INFO:Progress:3478/3976
INFO:Progress:3479/3976
INFO:Progress:3480/3976
INFO:Progress:3481/3976
INFO:Progress:3482/3976
INFO:Progress:3483/3976
INFO:Progress:3484/3976
INFO:Progress:3485/3976
INFO:Progress:3486/3976
INFO:Progress:3487/3976
INFO:Progress:3488/3976
INFO:Progress:3489/3976
INFO:Progress:3490/3976
INFO:Progress:3491/3976
INFO:Progress:3492/3976
INFO:Progress:3493/3976
INFO:Progress:3494/3976
INFO:Progress:3495/3976
INFO:Progress:3496/3976
INFO:Progress:3497/3976
INFO:Progress:3498/3976
INFO:Progress:3499/3976
INFO:Progress:3500/3976
INFO:Progress:3501/3976
INFO:Progress:3502/3976
INFO:Progress:3503/3976
INFO:Progress:3504/3976
INFO:Progress:3505/3976
INFO:Progress:3506/3976
INFO:Progress:3507/3976
INFO:Progress:35

INFO:Progress:3809/3976
INFO:Progress:3810/3976
INFO:Progress:3811/3976
INFO:Progress:3812/3976
INFO:Progress:3813/3976
INFO:Progress:3814/3976
INFO:Progress:3815/3976
INFO:Progress:3816/3976
INFO:Progress:3817/3976
INFO:Progress:3818/3976
INFO:Progress:3819/3976
INFO:Progress:3820/3976
INFO:Progress:3821/3976
INFO:Progress:3822/3976
INFO:Progress:3823/3976
INFO:Progress:3824/3976
INFO:Progress:3825/3976
INFO:Progress:3826/3976
INFO:Progress:3827/3976
INFO:Progress:3828/3976
INFO:Progress:3829/3976
INFO:Progress:3830/3976
INFO:Progress:3831/3976
INFO:Progress:3832/3976
INFO:Progress:3833/3976
INFO:Progress:3834/3976
INFO:Progress:3835/3976
INFO:Progress:3836/3976
INFO:Progress:3837/3976
INFO:Progress:3838/3976
INFO:Progress:3839/3976
INFO:Progress:3840/3976
INFO:Progress:3841/3976
INFO:Progress:3842/3976
INFO:Progress:3843/3976
INFO:Progress:3844/3976
INFO:Progress:3845/3976
INFO:Progress:3846/3976
INFO:Progress:3847/3976
INFO:Progress:3848/3976
INFO:Progress:3849/3976
INFO:Progress:38

In [18]:
prova = 'ilGabbrone'

print(max(viewerOverlapDict[prova], key=viewerOverlapDict[prova].get))
print(viewerOverlapDict[prova][max(viewerOverlapDict[prova], key=viewerOverlapDict[prova].get)])

n1sc_00
66.67
